In [26]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import root_mean_squared_error

import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

c:\Users\trinh\miniconda3\envs\mlenv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
import mlflow


In [4]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('my_new_exp')

<Experiment: artifact_location='/workspaces/mlops-project/03-tracking/mlruns/1', creation_time=1764778521062, experiment_id='1', last_update_time=1765012433941, lifecycle_stage='active', name='my_new_exp', tags={'mlflow.experimentKind': 'custom_model_development'}>

# Q1 


In [5]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
len(df.columns)

19

answer : 19

## Q2

In [6]:
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'])
df.duration = df.duration.apply(lambda x : x.total_seconds() / 60)

In [7]:
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

answer : 42.59

# Q3

In [8]:
# removing outliers

df_clean = df[(df.duration >= 1) & (df.duration <= 60)]
outliers = len(df) - len(df_clean)

In [9]:
1 - outliers / len(df)

0.9812202822125979

# Q4

In [10]:
categorical = ['PULocationID','DOLocationID']
df_clean[categorical] = df_clean[categorical].astype(str)

C:\Users\trinh\AppData\Local\Temp\ipykernel_12536\2023085547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[categorical] = df_clean[categorical].astype(str)


In [11]:
inputs = categorical + ['trip_distance']

In [12]:
dict_df = df_clean[categorical].to_dict(orient="records")

In [13]:
vectorizer = DictVectorizer()
X_train = vectorizer.fit_transform(dict_df)
y_train = df_clean.duration.values


# lr = LinearRegression()
# lr.fit(X_train,y_train)

# y_pred = lr.predict(X_train)

# root_mean_squared_error(y_pred,y_train)

# test

In [14]:
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet')
df_val.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [15]:
df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime'])
df_val.duration = df_val.duration.apply(lambda x : x.total_seconds() / 60)

# removing outliers

df_clean_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]
df_clean_val[categorical] = df_clean_val[categorical].astype(str)
dict_df_val = df_clean_val[categorical].to_dict(orient="records")

C:\Users\trinh\AppData\Local\Temp\ipykernel_12536\3965524849.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_val[categorical] = df_clean_val[categorical].astype(str)


In [16]:

X_val= vectorizer.transform(dict_df_val)
y_val= df_clean_val.duration.values

# y_pred = lr.predict(X_val)

# root_mean_squared_error(y_pred,y_val)

# mlflow

In [ ]:
# # lasso

# with mlflow.start_run():
#     mlflow.set_tag('developer', "nn")

#     mlflow.log_param('train-path','yellow_tripdata_2023-01.parquet')
#     mlflow.log_param('valid-path', 'yellow_tripdata_2023-02.parquet')

#     alpha = 0.1

#     mlflow.log_param("alpha", alpha)
#     lr2 = Lasso(alpha)
#     lr2.fit(X_train, y_train)

#     y_pred = lr2.predict(X_val)

#     rmse = root_mean_squared_error(y_pred,y_val)
#     mlflow.log_metric("rmse", rmse)


In [18]:
train = xgb.DMatrix(X_train ,label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [19]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, "validation")],
            early_stopping_rounds=20
        )
        y_pred=booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    
    return {"loss":rmse, "status": STATUS_OK}


In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 15, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.80568                           
[1]	validation-rmse:9.57562                           
[2]	validation-rmse:9.37454                           
[3]	validation-rmse:9.19677                           
[4]	validation-rmse:9.04269                           
[5]	validation-rmse:8.90822                           
[6]	validation-rmse:8.79043                           
[7]	validation-rmse:8.68909                           
[8]	validation-rmse:8.59680                           
[9]	validation-rmse:8.51773                           
[10]	validation-rmse:8.44876                          
[11]	validation-rmse:8.38409                          
[12]	validation-rmse:8.33206                          
[13]	validation-rmse:8.28520                          
[14]	validation-rmse:8.24291                          
[15]	validation-rmse:8.20538                          
[16]	validation-rmse:8.16670                          
[17]	validation-rmse:8.13779                          
[18]	valid

KeyboardInterrupt: 

In [ ]:
params = {
    'max_depth': 8,
    'learning_rate': 0.5713570775535193,
    'reg_alpha': 0.0102506342868148,
    'reg_lambda': 0.21151517251054874,
    'min_child_weight': 0.5421059866401473,
    'objective':'reg:squarederror',
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=500,
            evals=[(valid, "validation")],
            early_stopping_rounds=20
        )



2025/12/05 15:39:23 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '18b620ebc83948268ad2806b2aa7f982', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


KeyboardInterrupt: 

In [ ]:
mlflow.xgboost.autolog(disable=True)

## model management

In [ ]:
# lasso

with mlflow.start_run():
    mlflow.set_tag('developer', "nn")

    mlflow.log_param('train-path','yellow_tripdata_2023-01.parquet')
    mlflow.log_param('valid-path', 'yellow_tripdata_2023-02.parquet')

    alpha = 0.1

    mlflow.log_param("alpha", alpha)
    lr2 = Lasso(alpha)
    lr2.fit(X_train, y_train)

    y_pred = lr2.predict(X_val)

    rmse = root_mean_squared_error(y_pred,y_val)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_artifact(local_path="models/lasso.bin", artifact_path="models_pickle")


MlflowException: Invalid artifact path: 'models_pickle/'. Names may be treated as files in certain cases, and must not resolve to other names when treated as such. This name would resolve to 'models_pickle'

In [22]:
import pickle
with open('models/lasso.bin', 'wb') as f_out:
    pickle.dump((vectorizer, lr2), f_out)

### xgb

In [28]:
train = xgb.DMatrix(X_train ,label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [32]:
mlflow.xgboost.autolog(disable=True)

In [34]:

with mlflow.start_run():
    best_params = {
    'max_depth': 8,
    'learning_rate': 0.5713570775535193,
    'reg_alpha': 0.0102506342868148,
    'reg_lambda': 0.21151517251054874,
    'min_child_weight': 0.5421059866401473,
    'objective':'reg:squarederror',
    'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=500,
                evals=[(valid, "validation")],
                early_stopping_rounds=20
            )
    
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_pred, y_val)
    mlflow.log_metric('rmse', rmse)

    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(vectorizer, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor'")

    mlflow.xgboost.log_model(booster, artifact_path='mlflow')

[0]	validation-rmse:8.52896
[1]	validation-rmse:8.13898
[2]	validation-rmse:7.98556
[3]	validation-rmse:7.91210
[4]	validation-rmse:7.83655
[5]	validation-rmse:7.75183
[6]	validation-rmse:7.68909
[7]	validation-rmse:7.62649
[8]	validation-rmse:7.57978
[9]	validation-rmse:7.54755
[10]	validation-rmse:7.51373
[11]	validation-rmse:7.45237
[12]	validation-rmse:7.41655
[13]	validation-rmse:7.39845
[14]	validation-rmse:7.37142
[15]	validation-rmse:7.33413
[16]	validation-rmse:7.29776
[17]	validation-rmse:7.26233
[18]	validation-rmse:7.24822
[19]	validation-rmse:7.23433
[20]	validation-rmse:7.21941
[21]	validation-rmse:7.20600
[22]	validation-rmse:7.19390
[23]	validation-rmse:7.14767
[24]	validation-rmse:7.13716
[25]	validation-rmse:7.12330
[26]	validation-rmse:7.11424
[27]	validation-rmse:7.07061
[28]	validation-rmse:7.05037
[29]	validation-rmse:7.03646
[30]	validation-rmse:6.98035
[31]	validation-rmse:6.96745
[32]	validation-rmse:6.96057
[33]	validation-rmse:6.94089
[34]	validation-rmse:6.9

2025/12/06 10:47:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


# run logged model

In [50]:
import mlflow.pyfunc
RUN_ID = 'b05e5fc615af472d8ffc1326b8971e44'
# Charger le modèle depuis MLflow Tracking
model_uri = f"runs:/{RUN_ID}/mlflow"
model = mlflow.pyfunc.load_model(model_uri)

In [52]:
xgboost_model = mlflow.xgboost.load_model(model_uri)

In [54]:
y_pred = xgboost_model.predict(valid)

In [55]:
y_pred[10:]

array([20.052647 , 19.373825 ,  5.4323263, ..., 14.9079   , 11.456302 ,
       13.774116 ], shape=(2855941,), dtype=float32)